In [7]:
import pandas as pd
import numpy as np
import glob
import json
from tqdm import tqdm
import subprocess
from pathlib import Path

In [8]:
path = '/hdd/backup_cluster_august_2023/Self-Supervised-Learning-with-Predictability-Minimization/Self-Supervised-Learning-with-Predictability-Minimization/trained_models'

In [9]:
# find all folders in the path:
folders = glob.glob(path + '/*')
# the model name is the last part of the path:
model_names = [folder.split('/')[-1] for folder in folders]
# make a list of tuples, first the model name, second the path
model_paths = list(zip(model_names, folders))
# exclude folders with model_name 'barlow_cl_lin_pred_min' and 'cl_non_lin_pred_min'
model_paths = [model_path for model_path in model_paths if model_path[0] not in ['barlow_cl_lin_pred_min', 'cl_non_lin_pred_min']]
print(model_paths)

[('vicreg', '/hdd/backup_cluster_august_2023/Self-Supervised-Learning-with-Predictability-Minimization/Self-Supervised-Learning-with-Predictability-Minimization/trained_models/vicreg'), ('cl_lin_pred_min', '/hdd/backup_cluster_august_2023/Self-Supervised-Learning-with-Predictability-Minimization/Self-Supervised-Learning-with-Predictability-Minimization/trained_models/cl_lin_pred_min'), ('mocov3', '/hdd/backup_cluster_august_2023/Self-Supervised-Learning-with-Predictability-Minimization/Self-Supervised-Learning-with-Predictability-Minimization/trained_models/mocov3'), ('barlow_twins', '/hdd/backup_cluster_august_2023/Self-Supervised-Learning-with-Predictability-Minimization/Self-Supervised-Learning-with-Predictability-Minimization/trained_models/barlow_twins'), ('vibcreg', '/hdd/backup_cluster_august_2023/Self-Supervised-Learning-with-Predictability-Minimization/Self-Supervised-Learning-with-Predictability-Minimization/trained_models/vibcreg'), ('byol', '/hdd/backup_cluster_august_2023/

In [10]:
models_to_process = []

# now we loop over all model paths
for model_name, folder in model_paths:
    # find all subfolders:
    subfolders = glob.glob(folder + '/*')
    # print(subfolders)
    # iterate over all subfolder
    for subfolder in subfolders:
        # check out the args.json in the subfolder
        args = json.load(open(subfolder + '/args.json'))
        # find the dataset key
        dataset = args['data']['dataset']
        max_epochs = args['max_epochs']
        run_id = args['wandb_run_id']
        # check if there is a file in the subfolder that ends with  '{max_epochs-1}.ckpt', if so extract its path
        possible_file = glob.glob(subfolder + f'/*{max_epochs-1}.ckpt')
        # print(posible_file)
        assert len(possible_file) <= 1
        if len(possible_file) == 1:
            models_to_process.append((run_id, model_name, dataset, max_epochs, possible_file[0]))
print(models_to_process)

[('pxqwg3ex', 'vicreg', 'imagenet100', 400, '/hdd/backup_cluster_august_2023/Self-Supervised-Learning-with-Predictability-Minimization/Self-Supervised-Learning-with-Predictability-Minimization/trained_models/vicreg/pxqwg3ex/vicreg-imagenet100-pxqwg3ex-ep=399.ckpt'), ('000hudwm', 'vicreg', 'cifar100', 1000, '/hdd/backup_cluster_august_2023/Self-Supervised-Learning-with-Predictability-Minimization/Self-Supervised-Learning-with-Predictability-Minimization/trained_models/vicreg/000hudwm/vicreg-cifar100-000hudwm-ep=999.ckpt'), ('31yfnww4', 'vicreg', 'cifar100', 1000, '/hdd/backup_cluster_august_2023/Self-Supervised-Learning-with-Predictability-Minimization/Self-Supervised-Learning-with-Predictability-Minimization/trained_models/vicreg/31yfnww4/vicreg-cifar100-31yfnww4-ep=999.ckpt'), ('ihgpptnq', 'vicreg', 'imagenet100', 400, '/hdd/backup_cluster_august_2023/Self-Supervised-Learning-with-Predictability-Minimization/Self-Supervised-Learning-with-Predictability-Minimization/trained_models/vicr

In [11]:
# for every single entry we now generate the train and test embeddings and the correlation files
'''
A sample call looks as follows:
python3 main_corr.py \
    --dataset cifar10 \
    --batch_size 16 \
    --num_workers 10 \
    --train_data_path "./datasets" \
    --val_data_path "./datasets" \
    --pretrained_checkpoint_dir $1 \
    --pretrained_checkpoint $2
'''

data_paths = {
    'cifar10': {
        'train': "./datasets",
        'val': "./datasets",
        },
    'cifar100':  {
        'train': "./datasets",
        'val': "./datasets",
        },
    'imagenet100': {
        'train': '/home/david/data/ILSVRC2012_img_train_100',
        'val': '/home/david/data/ILSVRC2012_img_val_100',
        },
}

for run_id, model_name, dataset, max_epochs, model_path in tqdm(models_to_process):
    # pretrained_checkpoint_dir is the folder the model in modelpath is in
    pretrained_checkpoint_dir = Path(model_path).parent
    # now we run the command
    print(pretrained_checkpoint_dir)
    print(model_path)
    subprocess.run(['python3', '../main_corr.py', '--dataset', dataset, '--batch_size', '128', '--num_workers', '10', '--train_data_path', data_paths[dataset]['train'], '--val_data_path', data_paths[dataset]['val'], '--pretrained_checkpoint_dir', pretrained_checkpoint_dir, '--pretrained_checkpoint', model_path])


  0%|          | 0/62 [00:00<?, ?it/s]

/hdd/backup_cluster_august_2023/Self-Supervised-Learning-with-Predictability-Minimization/Self-Supervised-Learning-with-Predictability-Minimization/trained_models/vicreg/pxqwg3ex
/hdd/backup_cluster_august_2023/Self-Supervised-Learning-with-Predictability-Minimization/Self-Supervised-Learning-with-Predictability-Minimization/trained_models/vicreg/pxqwg3ex/vicreg-imagenet100-pxqwg3ex-ep=399.ckpt


/home/david/.local/share/virtualenvs/Self-Supervised-Learning-with-Predictabili-RPXJKYD0/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/david/.local/share/virtualenvs/Self-Supervised-Learning-with-Predictabili-RPXJKYD0/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-

{'name': 'vicreg-imagenet100', 'method': 'vicreg', 'backbone': {'name': 'resnet18', 'kwargs': {}}, 'method_kwargs': {'proj_hidden_dim': 2048, 'proj_output_dim': 2048, 'sim_loss_weight': 25.0, 'var_loss_weight': 25.0, 'cov_loss_weight': 1.0}, 'data': {'dataset': 'imagenet100', 'train_path': '/itet-stor/zdavid/net_scratch/data/ILSVRC2012_img_train_100', 'val_path': '/itet-stor/zdavid/net_scratch/data/ILSVRC2012_img_val_100', 'format': 'dali', 'num_workers': 4, 'no_labels': False, 'fraction': -1, 'num_classes': 100, 'num_large_crops': 2, 'num_small_crops': 0}, 'augmentations': [{'rrc': {'enabled': True, 'crop_min_scale': 0.2, 'crop_max_scale': 1.0}, 'color_jitter': {'prob': 0.8, 'brightness': 0.4, 'contrast': 0.4, 'saturation': 0.2, 'hue': 0.1}, 'grayscale': {'prob': 0.2}, 'gaussian_blur': {'prob': 0.5}, 'solarization': {'prob': 0.1}, 'equalization': {'prob': 0.0}, 'horizontal_flip': {'prob': 0.5}, 'crop_size': 224, 'num_crops': 2}], 'optimizer': {'name': 'lars', 'batch_size': 128, 'lr': 

{'name': 'vicreg-imagenet100', 'method': 'vicreg', 'backbone': {'name': 'resnet18', 'kwargs': {}}, 'method_kwargs': {'proj_hidden_dim': 2048, 'proj_output_dim': 2048, 'sim_loss_weight': 25.0, 'var_loss_weight': 25.0, 'cov_loss_weight': 1.0}, 'data': {'dataset': 'imagenet100', 'train_path': '/itet-stor/zdavid/net_scratch/data/ILSVRC2012_img_train_100', 'val_path': '/itet-stor/zdavid/net_scratch/data/ILSVRC2012_img_val_100', 'format': 'dali', 'num_workers': 4, 'no_labels': False, 'fraction': -1, 'num_classes': 100, 'num_large_crops': 2, 'num_small_crops': 0}, 'augmentations': [{'rrc': {'enabled': True, 'crop_min_scale': 0.2, 'crop_max_scale': 1.0}, 'color_jitter': {'prob': 0.8, 'brightness': 0.4, 'contrast': 0.4, 'saturation': 0.2, 'hue': 0.1}, 'grayscale': {'prob': 0.2}, 'gaussian_blur': {'prob': 0.5}, 'solarization': {'prob': 0.1}, 'equalization': {'prob': 0.0}, 'horizontal_flip': {'prob': 0.5}, 'crop_size': 224, 'num_crops': 2}], 'optimizer': {'name': 'lars', 'batch_size': 128, 'lr': 

...

/hdd/backup_cluster_august_2023/Self-Supervised-Learning-with-Predictability-Minimization/Self-Supervised-Learning-with-Predictability-Minimization/trained_models/simclr/5dgztcwi
/hdd/backup_cluster_august_2023/Self-Supervised-Learning-with-Predictability-Minimization/Self-Supervised-Learning-with-Predictability-Minimization/trained_models/simclr/5dgztcwi/simclr-cifar100-5dgztcwi-ep=999.ckpt


/home/david/.local/share/virtualenvs/Self-Supervised-Learning-with-Predictabili-RPXJKYD0/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/david/.local/share/virtualenvs/Self-Supervised-Learning-with-Predictabili-RPXJKYD0/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-

{'name': 'simclr-cifar100', 'method': 'simclr', 'backbone': {'name': 'resnet18', 'kwargs': {}}, 'method_kwargs': {'proj_hidden_dim': 2048, 'proj_output_dim': 256, 'temperature': 0.2}, 'data': {'dataset': 'cifar100', 'train_path': './datasets', 'val_path': 'datasets/imagenet100/val', 'format': 'image_folder', 'num_workers': 4, 'no_labels': False, 'fraction': -1, 'num_classes': 100, 'num_large_crops': 2, 'num_small_crops': 0}, 'optimizer': {'name': 'lars', 'batch_size': 256, 'lr': 0.4, 'classifier_lr': 0.1, 'weight_decay': 0.0001, 'kwargs': {'clip_lr': True, 'eta': 0.02, 'exclude_bias_n_norm': True, 'momentum': 0.9}, 'exclude_bias_n_norm_wd': False}, 'scheduler': {'name': 'warmup_cosine', 'lr_decay_steps': None, 'min_lr': 0.0, 'warmup_start_lr': 3e-05, 'warmup_epochs': 10, 'interval': 'step'}, 'checkpoint': {'enabled': True, 'dir': 'trained_models', 'frequency': 1, 'keep_prev': False}, 'auto_resume': {'enabled': True, 'max_hours': 36}, 'max_epochs': 1000, 'devices': [0], 'sync_batchnorm'

Files already downloaded and verified
Files already downloaded and verified
{'name': 'simclr-cifar100', 'method': 'simclr', 'backbone': {'name': 'resnet18', 'kwargs': {}}, 'method_kwargs': {'proj_hidden_dim': 2048, 'proj_output_dim': 256, 'temperature': 0.2}, 'data': {'dataset': 'cifar100', 'train_path': './datasets', 'val_path': 'datasets/imagenet100/val', 'format': 'image_folder', 'num_workers': 4, 'no_labels': False, 'fraction': -1, 'num_classes': 100, 'num_large_crops': 2, 'num_small_crops': 0}, 'optimizer': {'name': 'lars', 'batch_size': 256, 'lr': 0.4, 'classifier_lr': 0.1, 'weight_decay': 0.0001, 'kwargs': {'clip_lr': True, 'eta': 0.02, 'exclude_bias_n_norm': True, 'momentum': 0.9}, 'exclude_bias_n_norm_wd': False}, 'scheduler': {'name': 'warmup_cosine', 'lr_decay_steps': None, 'min_lr': 0.0, 'warmup_start_lr': 3e-05, 'warmup_epochs': 10, 'interval': 'step'}, 'checkpoint': {'enabled': True, 'dir': 'trained_models', 'frequency': 1, 'keep_prev': False}, 'auto_resume': {'enabled': 

100%|██████████| 62/62 [1:11:41<00:00, 69.37s/it]
